In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In this notebook, we can estimate: how many visits could we have for every field, if we evenly split a typical total number of visits over a very large sky footprint? 

So first: estimate the total number of visits available. 

In [ ]:
# Total approximate number of visits in opsim:
totalNvis = 2400000
# Let's say we can play with 93% of these visits
totalNvis *= 0.93
print("The number of visits available is %d (%.2fM)" % (totalNvis, totalNvis/1000000))

Now we'll count the number of fields we want to include in this 'big sky' footprint, using the opsim tessellation. This is equivalent to estimating the number of pointings required to cover the footprint.

In [ ]:
# Read the field list.
fields = pd.read_csv('field_list.csv')
# what does the field list look like?
fields[0:5]

In [ ]:
# Select fields with declination between -90 and 32 -- our "big sky" footprint
bigsky = fields.query('(dec >= -90) and (dec <= 32)') 

In [ ]:
# Plot the fields so we can check what this footprint looks like.
def radec2project(ra, dec):
    return (np.radians(ra) - np.pi, np.radians(dec))

fig = plt.figure(figsize=(8, 8))
ax = plt.subplot(111, projection="aitoff")
x, y = radec2project(bigsky.ra, bigsky.dec)
ax.scatter(x, y)
plt.grid(True)

In [ ]:
# How many fields are included in this "big sky" version of the survey?
nfields = len(bigsky)
print("The number of fields in the footprint is %d" % (nfields))

So how many visits could we potentially have per field, if we split them evenly among fields?

In [ ]:
# Exact number of visits per field
visPerField = totalNvis / nfields
# Round the number of visits per field to an integer
visPerField = int(round(visPerField))
# And recalculate what this means for the totalNvis -- if the total # changed too much, maybe round down
propTotal = visPerField * nfields
print("Per field, this corresponds to %d visits" % (visPerField, ))
print("This implies a total number of visits required of %d (compared to original estimate of %d): %.2f%s change"
     % (propTotal, totalNvis, float(propTotal - totalNvis) / totalNvis * 100.0, "%"))